# Bank account fraud detection
Daniel Mizrahi (10675418), Antonio La Chira Marquina (11847018)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from read_data import read_dataset

## Parsing the data
We first start off by reading in the data. Since some of the data is categorical and therefore non-numerical we must use one hot encoding to process the data and make sure this won't be a problem when implenting our classifiers. Altough perhaps marginal when converting categorical data we drop the first column so there is less data later on to perform calculations on. The original data without one hot encoding can be used to make conclusions about the data

In [ ]:
from read_data import read_dataset

data_original = read_dataset('datasets/Base.csv', one_hot_encoding=False)
data = read_dataset('datasets/Base.csv', one_hot_encoding=True, drop_first=True)

In [ ]:
print('Preview of the one hot encoded data in table form (first 10 rows and 4 columns):')
print(data.head(10).iloc[:, :4])

print(f'\nOrignal data shape (rows, columns): {data_original.shape}')
print(f'One hot encoded data shape (rows, columns): {data.shape}')

In [ ]:
# k = 'auto'
k = 50

categorical_data = ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
boolean_data = ['fraud_bool', 'email_is_free', 'phone_home_valid', 'phone_mobile_valid', 'has_other_cards', 'foreign_request', 'keep_alive_sessions', 'device_dinstinct_emails_8w', 'device_fraud_count']

fig, axs = plt.subplots(16, 2, figsize=(16,100))
column_names = list(data_original.columns.values)
for column, ax in enumerate(axs.flat):
    ax.set_xlabel(column_names[column])
    ax.set_ylabel('Count')
    if column_names[column] in boolean_data:
        ax.hist(data_original.iloc[:, column], 2)
    elif column_names[column] in categorical_data:
        ax.hist(data_original.iloc[:, column], data_original.iloc[:, column].nunique())
    else:
        ax.hist(data_original.iloc[:, column], k)
    if column_names[column] not in categorical_data + boolean_data:
        mean = np.mean(data_original.iloc[:, column])
        standard_deviation = np.std(data_original.iloc[:, column])
        ax.axvline(mean, color='g')
        if mean - standard_deviation > np.amin(data_original.iloc[:, column]):
            ax.axvline(mean - standard_deviation, color='r')
        ax.axvline(mean + standard_deviation, color='r')
        ax.legend([f'mean (approx {round(mean, 2)})', f'standard deviation (approx {round(standard_deviation, 2)})'])

In [ ]:
# k = 'auto'
k = 50

categorical_data = ['payment_type', 'employment_status', 'housing_status', 'source', 'device_os']
boolean_data = ['fraud_bool', 'email_is_free', 'phone_home_valid', 'phone_mobile_valid', 'has_other_cards', 'foreign_request', 'keep_alive_sessions', 'device_dinstinct_emails_8w', 'device_fraud_count']

data_fraud = data_original.loc[data_original['fraud_bool'] == 1]
data_no_fraud = data_original.loc[data_original['fraud_bool'] == 0]

fig, axs = plt.subplots(16, 2, figsize=(16,100))
column_names = list(data_original.columns.values)
for column, ax in enumerate(axs.flat):
    ax.set_xlabel(column_names[column])
    ax.set_ylabel('Count')
    if column_names[column] in boolean_data:
        ax.hist(data_no_fraud.iloc[:, column], 2, label='No fraud')
        ax.hist(data_fraud.iloc[:, column], 2, color='y', label='Fraud')
    elif column_names[column] in categorical_data:
        ax.hist(data_no_fraud.iloc[:, column], data_fraud.iloc[:, column].nunique(), label='No fraud')
        ax.hist(data_fraud.iloc[:, column], data_fraud.iloc[:, column].nunique(), color='y', label='Fraud')
    else:
        ax.hist(data_no_fraud.iloc[:, column], k, label='No fraud')
        ax.hist(data_fraud.iloc[:, column], k, color='y', label='Fraud')
    ax.legend()
    # Men moet uitzoeken hoe de legend hierboven gemerged kan worden met die in de if
    if column_names[column] not in categorical_data + boolean_data:
        _, labels = ax.get_legend_handles_labels()
        mean = np.mean(data_fraud.iloc[:, column])
        standard_deviation = np.std(data_fraud.iloc[:, column])
        ax.axvline(mean, color='g')
        if mean - standard_deviation > np.amin(data_fraud.iloc[:, column]):
            ax.axvline(mean - standard_deviation, color='r')
        ax.axvline(mean + standard_deviation, color='r')
        ax.legend([f'mean (approx {round(mean, 2)})', f'standard deviation (approx {round(standard_deviation, 2)})'])